## FILE CREATION

**GOAL**:
In this notebook we will create a **parquet** file (more efficient than csv) combining data from NO2 levels from all the months in the year 2018. We will also join two more tables containing info from the stations and info from the temperatures.

In [119]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

from pathlib import Path

### Reading the csv we will use

In [120]:
path = '/home/carleondel/data-madrid_pollution_project/'

In [121]:
df_stations = pd.read_csv(path + 'madrid_air_quality_stations.csv')
df_temp = pd.read_csv(path + 'madrid_hourly_temperatures_2018.csv')

In [122]:

# Define a list of month names
months = ['ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dic']

# Create an empty dictionary to store DataFrames
monthly_dfs = {}

# Loop through the months and read corresponding CSV files
for month in months:
    file_path = path + f'{month}_mo18.csv'
    df_month = pd.read_csv(file_path, sep=';')
    # Store the DataFrame in the dictionary with the month name as the key
    monthly_dfs[month] = df_month


Now for every month, we:
1. Filter by NO2 levels (MAGNITUD = 8)  (and drop that column)
2. Change the 0 measurements (Not Valid measurements) into NaN
3. Convert the hour columns into rows
4. Create a date 'FECHA' column (and drop all the date related features)

In [123]:
filtered_monthly_dfs = {}

# List of columns starting with 'H'
H_cols = [f'H{i:02d}' for i in range(1, 25)]

# Loop through the months
for month in months:
    # Access the DataFrame for the current month
    df = monthly_dfs[month]
    
    # Filter the DataFrame to keep only NO2 levels
    df_NO2 = df[df['MAGNITUD'] == 8]
    
    # Drop the 'MAGNITUD' column
    df_NO2 = df_NO2.drop('MAGNITUD', axis=1)

    # Replace 0 values with NaN in the selected columns
    df_NO2[H_cols] = df_NO2[H_cols].replace(0, np.nan)

    # Melt to convert the hour columns into rows
    melted_df = pd.melt(df_NO2, id_vars=['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA'], value_vars=H_cols, var_name='HORA', value_name='NO2_level')

    # Create new column with the last 2 positions from HORA
    # Then we substract 1 and add half hour (this way we avoid trouble with hour 24)
    # and convert it to str
    melted_df['HORA_new'] = (melted_df['HORA'].astype(str).str[-2:].astype(int) - 1).astype(str) + ':30:00'

    # Combine 'ANO', 'MES', 'DIA', and 'HORA' columns to form the 'FECHA' column
    melted_df['FECHA'] = pd.to_datetime(melted_df[['ANO', 'MES', 'DIA']].astype(str).agg('-'.join, axis=1) + ' ' + melted_df['HORA_new'].astype(str), format='%Y-%m-%d %H:%M:%S')
    
    # Drop unnecessary columns
    melted_df = melted_df.drop(columns=['ANO', 'MES', 'DIA', 'HORA', 'HORA_new'])

    # Store the filtered DataFrame in the dictionary
    filtered_monthly_dfs[month] = melted_df

In [124]:
filtered_monthly_dfs['ene']

,PROVINCIA,MUNICIPIO,ESTACION,PUNTO_MUESTREO,NO2_level,FECHA
0,28,79,4,28079004_8_8,15.0,2018-01-01 00:30:00
1,28,79,4,28079004_8_8,13.0,2018-01-02 00:30:00
2,28,79,4,28079004_8_8,35.0,2018-01-03 00:30:00
3,28,79,4,28079004_8_8,35.0,2018-01-04 00:30:00
4,28,79,4,28079004_8_8,24.0,2018-01-05 00:30:00
...,...,...,...,...,...,...
17851,28,79,60,28079060_8_8,19.0,2018-01-27 23:30:00
17852,28,79,60,28079060_8_8,20.0,2018-01-28 23:30:00
17853,28,79,60,28079060_8_8,85.0,2018-01-29 23:30:00
17854,28,79,60,28079060_8_8,73.0,2018-01-30 23:30:00


Let's prepare our dfs containing info from temperature and stations to join them

In [125]:
df_temp.head()

,date,hour,temp
0,2018-01-01,0,7.3
1,2018-01-01,1,7.4
2,2018-01-01,2,6.8
3,2018-01-01,3,7.1
4,2018-01-01,4,5.3


In [126]:
# Combine 'date' and 'hour' columns and add '30:00'
df_temp['date'] = pd.to_datetime(df_temp['date'] + ' ' + df_temp['hour'].astype(str) + ':30:00', format='%Y-%m-%d %H:%M:%S')

# Drop the 'hour' column
df_temp = df_temp.drop(['hour'], axis=1)

In [127]:
df_temp.head()

,date,temp
0,2018-01-01 00:30:00,7.3
1,2018-01-01 01:30:00,7.4
2,2018-01-01 02:30:00,6.8
3,2018-01-01 03:30:00,7.1
4,2018-01-01 04:30:00,5.3


Now our temperatures dataframe is ready to be joined with our months using date as key

In [128]:
df_stations.head()

,station,area,name,address,altitude,type,longitude,latitude
0,28079004,centro,Pza. de España,Plaza de España,635,UT,-3.712247,40.423853
1,28079008,centro,Escuelas Aguirre,Entre C/ Alcalá y C/ O' Donell,670,UT,-3.682319,40.421564
2,28079011,centro,Avda. Ramón y Cajal,Avda. Ramón y Cajal esq. C/ Príncipe de Vergara,708,UT,-3.677356,40.451475
3,28079016,noreste,Arturo Soria,C/ Arturo Soria esq. C/ Vizconde de los Asilos,693,UF,-3.639233,40.440047
4,28079017,suroeste,Villaverde,C/. Juan Peñalver,604,UF,-3.713322,40.347139


In [129]:
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   station    24 non-null     int64  
 1   area       24 non-null     object 
 2   name       24 non-null     object 
 3   address    24 non-null     object 
 4   altitude   24 non-null     int64  
 5   type       24 non-null     object 
 6   longitude  24 non-null     float64
 7   latitude   24 non-null     float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.6+ KB


To join this stations table, we need to modify first our monthly dataframes so 'PUNTO_MUESTREO' has the same format as 'station'

In [130]:
ready_monthly_dfs = {}


# Loop through the months
for month in months:
    # Access the DataFrame for the current month
    df = filtered_monthly_dfs[month]
    df['PUNTO_MUESTREO'] = (df['PUNTO_MUESTREO'].str[:-4]).astype(int)
    
    # Store the filtered DataFrame in the dictionary
    ready_monthly_dfs[month] = df

In [131]:
ready_monthly_dfs['ene'].head()

,PROVINCIA,MUNICIPIO,ESTACION,PUNTO_MUESTREO,NO2_level,FECHA
0,28,79,4,28079004,15.0,2018-01-01 00:30:00
1,28,79,4,28079004,13.0,2018-01-02 00:30:00
2,28,79,4,28079004,35.0,2018-01-03 00:30:00
3,28,79,4,28079004,35.0,2018-01-04 00:30:00
4,28,79,4,28079004,24.0,2018-01-05 00:30:00


### Everything is set to be joined
- First we will concat all our month tables
- Then we will left join our temp table based on 
- Final left join of our stations table

In [132]:
# Concatenate DataFrames
df_2018 = pd.concat(ready_monthly_dfs, ignore_index=True)


In [133]:
len(df_2018)

210000

In [134]:
merged_df = pd.merge(df_2018, df_temp, left_on='FECHA', right_on='date', how='left')
merged_df = merged_df.drop('date', axis=1)

In [135]:
merged_df.head()

,PROVINCIA,MUNICIPIO,ESTACION,PUNTO_MUESTREO,NO2_level,FECHA,temp
0,28,79,4,28079004,15.0,2018-01-01 00:30:00,7.3
1,28,79,4,28079004,13.0,2018-01-02 00:30:00,9.1
2,28,79,4,28079004,35.0,2018-01-03 00:30:00,7.5
3,28,79,4,28079004,35.0,2018-01-04 00:30:00,8.1
4,28,79,4,28079004,24.0,2018-01-05 00:30:00,9.7


In [138]:
df = pd.merge(merged_df, df_stations, left_on='PUNTO_MUESTREO', right_on='station', how='left')
df = df.drop('station', axis=1)

## Clean Dataframe

In [139]:
df.head()

,PROVINCIA,MUNICIPIO,ESTACION,PUNTO_MUESTREO,NO2_level,FECHA,temp,area,name,address,altitude,type,longitude,latitude
0,28,79,4,28079004,15.0,2018-01-01 00:30:00,7.3,centro,Pza. de España,Plaza de España,635,UT,-3.712247,40.423853
1,28,79,4,28079004,13.0,2018-01-02 00:30:00,9.1,centro,Pza. de España,Plaza de España,635,UT,-3.712247,40.423853
2,28,79,4,28079004,35.0,2018-01-03 00:30:00,7.5,centro,Pza. de España,Plaza de España,635,UT,-3.712247,40.423853
3,28,79,4,28079004,35.0,2018-01-04 00:30:00,8.1,centro,Pza. de España,Plaza de España,635,UT,-3.712247,40.423853
4,28,79,4,28079004,24.0,2018-01-05 00:30:00,9.7,centro,Pza. de España,Plaza de España,635,UT,-3.712247,40.423853


In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210000 entries, 0 to 209999
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   PROVINCIA       210000 non-null  int64         
 1   MUNICIPIO       210000 non-null  int64         
 2   ESTACION        210000 non-null  int64         
 3   PUNTO_MUESTREO  210000 non-null  int64         
 4   NO2_level       209676 non-null  float64       
 5   FECHA           210000 non-null  datetime64[ns]
 6   temp            210000 non-null  float64       
 7   area            210000 non-null  object        
 8   name            210000 non-null  object        
 9   address         210000 non-null  object        
 10  altitude        210000 non-null  int64         
 11  type            210000 non-null  object        
 12  longitude       210000 non-null  float64       
 13  latitude        210000 non-null  float64       
dtypes: datetime64[ns](1), float64(4), in

In [141]:
df.isna().sum()

PROVINCIA           0
MUNICIPIO           0
ESTACION            0
PUNTO_MUESTREO      0
NO2_level         324
FECHA               0
temp                0
area                0
name                0
address             0
altitude            0
type                0
longitude           0
latitude            0
dtype: int64

We save our clean dataframe into a parquet file

In [143]:
df.to_parquet('clean_data.parquet', index=False)